In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import layers, models
import kagglehub
import os
import pathlib
from sklearn.model_selection import train_test_split

c:\Users\luizc\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#CARREGANDO E PREPARANDO O DATASET
path = kagglehub.dataset_download("rakibuleceruet/drowsiness-prediction-dataset")
data_dir = pathlib.Path(os.path.join(path, '0 FaceImages'))

active_paths = list(data_dir.glob('Active Subjects/*.jpg'))
active_labels = [0] * len(active_paths) # 0 para 'alerta'
fatigue_paths = list(data_dir.glob('Fatigue Subjects/*.jpg'))
fatigue_labels = [1] * len(fatigue_paths) # 1 para 'fadiga'

all_image_paths = [str(p) for p in (active_paths + fatigue_paths)]
all_labels = active_labels + fatigue_labels

train_paths, val_paths, train_labels, val_labels = train_test_split(
    all_image_paths, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    return image, label

train_dataset = train_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE).cache().shuffle(len(train_paths)).batch(BATCH_SIZE).prefetch(AUTOTUNE)
validation_dataset = val_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).cache().prefetch(AUTOTUNE)


In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
], name="data_augmentation")

# CARREGA O MODELO BASE MOBILENETV2 PRÉ-TREINADO
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False # CONGELANDO OS PESOS DO MODELO

# CAMADAS DE CLASSIFICAÇÃO 
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_tl = tf.keras.Model(inputs, outputs) 

print("Modelo de Transfer Learning pronto")

model_tl.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\nIniciando o treinamento do modelo de Transfer Learning...")
history_tl = model_tl.fit(
    train_dataset,
    epochs=15,
    validation_data=validation_dataset
)

print("\n--- Avaliação Final do Modelo de Transfer Learning ---")
val_loss_tl, val_acc_tl = model_tl.evaluate(validation_dataset, verbose=2)
print(f"Acurácia final no conjunto de validação: {val_acc_tl * 100:.2f}%")

Modelo de Transfer Learning pronto

Iniciando o treinamento do modelo de Transfer Learning...
Epoch 1/15
228/228 ━━━━━━━━━━━━━━━━━━━━ 67s 250ms/step - accuracy: 0.7416 - loss: 0.4947 - val_accuracy: 0.7944 - val_loss: 0.4455
Epoch 2/15
228/228 ━━━━━━━━━━━━━━━━━━━━ 55s 240ms/step - accuracy: 0.7876 - loss: 0.4181 - val_accuracy: 0.8021 - val_loss: 0.4293
Epoch 3/15
228/228 ━━━━━━━━━━━━━━━━━━━━ 57s 249ms/step - accuracy: 0.8067 - loss: 0.3924 - val_accuracy: 0.8065 - val_loss: 0.3976
Epoch 4/15
228/228 ━━━━━━━━━━━━━━━━━━━━ 56s 248ms/step - accuracy: 0.8121 - loss: 0.3808 - val_accuracy: 0.8158 - val_loss: 0.3996
Epoch 5/15
228/228 ━━━━━━━━━━━━━━━━━━━━ 54s 238ms/step - accuracy: 0.8188 - loss: 0.3744 - val_accuracy: 0.8130 - val_loss: 0.4029
Epoch 6/15
228/228 ━━━━━━━━━━━━━━━━━━━━ 55s 241ms/step - accuracy: 0.8235 - loss: 0.3679 - val_accuracy: 0.8240 - val_loss: 0.3724
Epoch 7/15
228/228 ━━━━━━━━━━━━━━━━━━━━ 50s 219ms/step - accuracy: 0.8270 - loss: 0.3655 - val_accuracy: 0.8295 - val_lo

In [5]:
import pickle

MODELS_DIR = "saved_models"
HISTORIES_DIR = "historicos"
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(HISTORIES_DIR, exist_ok=True)   

model_tl_path = os.path.join(MODELS_DIR, 'modelo_transfer_learning.keras')
model_tl.save(model_tl_path)

loaded_model = tf.keras.models.load_model(model_tl_path)

#EXTRATOR DE CARACTERÍSTICAS

feature_extractor = tf.keras.Model(
    inputs=loaded_model.input,
    outputs=loaded_model.get_layer('global_average_pooling2d').output
    )

history_tl_path = os.path.join(HISTORIES_DIR, 'history_transfer_learning.pkl')

with open(history_tl_path, 'wb') as file:
    pickle.dump(history_tl.history, file)
    
print("modelo e histórico de treinamento salvos")

feature_extractor.summary()

modelo e histórico de treinamento salvos


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)